In [1]:
# 라이브러리
import ParsingNote as pn
import matplotlib.pyplot as plot
from scipy.io import wavfile
from pydub import AudioSegment

# file list 받기용
from os import listdir
from os.path import isfile, join

# print clear용
from IPython.display import clear_output

# 시간값에 따른 index 구하기
def find_index(audio, sample_rate, t):
    
    n = len(audio)
    start = 0.0
    end = len(audio) / sample_rate
    i = round((n-1) * (t - start) / (end - start))
    
    return int(i)

# 오디오 자르는 함수 - 오디오, 샘플레이트, 시작시간, 얼마 초 동안
def slicing_audio(audio, sample_rate, start_time, end_time):
    
    start_index = find_index(audio, sample_rate, start_time)
    end_index = find_index(audio, sample_rate, end_time)
    new_audio = audio[start_index:end_index].copy()
    
    return new_audio

# Spectrogram image 만드는 함수
def save_spectrogram(fil_name, sample_rate, audio, size):
    #plot 초기화
    fig,ax = plot.subplots(1)
    fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
    
    #크기설정
    fig.set_figheight(size)
    fig.set_figwidth(size)
    
    #Axis 끄기 및 spectrogram 생성
    ax.axis('off')
    pxx, freqs, bins, im = ax.specgram(x=audio, Fs=sample_rate, noverlap=384, NFFT=512)
    ax.axis('off')
    
    #저장
    fig.savefig(fil_name, dpi=100, frameon='false')
    
    #메모리 해제
    plot.cla()
    plot.clf()
    plot.close('all')
    
# Note Path 구하기(오디오 이름으로)
def FindNotePath(audio_name):
    note_root_path = "./Note/"
    note_name = ""

    for file in listdir(note_root_path):
        if audio_name in file:
            note_name = note_root_path + file
            break
    
    return note_name

# Stereo to Mono
def StereoToMono(audio_path, output_name):
    temp_audio_path = "./TempAudio/" + output_name + ".wav"
    temp_audio = AudioSegment.from_wav(audio_path)
    temp_audio = temp_audio.set_channels(1)
    temp_audio.export(temp_audio_path, format="wav")
    return temp_audio_path

D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# 스펙트로그램 이미지 생성
def MakeSpectrogramImages(audio_path, slice_unit, image_size):
    # 오디오 이름 구하기
    audio_name = audio_path.split('/')[-1].split('.')[0]
    
    # Note 불러오기
    note_path = FindNotePath(audio_name)
    parsing_note = pn.ParsingNoteFile(note_path)

    # Spectrogram 저장 폴더 경로
    dir_path = "./OutputColor/" + audio_name + "_Spectrogram"
    pn.MakeOutputDir(dir_path)
    
    # Stereo to Mono
    mono_path = StereoToMono(audio_path, "temp_audio")

    # sample_rate, data 읽기
    sample_rate, audio_data = wavfile.read(mono_path)

    # 총 시간 및 갯수
    total_time = len(audio_data) / sample_rate
    num = int(total_time / slice_unit)

    # 전체 step 출력
    print(audio_name + " step num : ",  num)
            
    # 오디오 잘라서 스펙트로그램 만들기
    for i in range(0, num):
        #오디오 자르기
        start_time = slice_unit * float(i)
        end_time = slice_unit * (float(i) + 1.0)
        new_audio = slicing_audio(audio_data, sample_rate, start_time, end_time)
    
        #이미지 분류 - 노트에 따라 파일 경로 구하기)
        file_path = ""
        image_num = str(i).zfill(4)
    
        if image_num in parsing_note:
            file_path = "".join([dir_path, '/', parsing_note[image_num], '/', audio_name, '_', image_num, '_output.jpg'])
        else:
            file_path = "".join([dir_path, '/N/', audio_name, '_', image_num, '_output.jpg'])
        
        save_spectrogram(file_path, sample_rate, new_audio, image_size)
    
        # Step 출력
        if i % 100 == 0:
            print("step : ", i)
    
    print(audio_name + " spectrograms done")

In [3]:
# output 설정
SLICE_VALUE = 0.1
IMAGE_SIZE = 1.28

#오디오 path 구하기
audio_root_path = "./Audio/"
input_paths = list()

for file in listdir(audio_root_path):
    file_path = join(audio_root_path, file)
    if isfile(file_path):
        input_paths.append(file_path)

# 총 갯수
total_audio_num = len(input_paths)
# 프로세스 진행도
show_process = 0

# 경로에 있는 모든 오디오 Spectrogram으로 출력
for path in input_paths:
    #프로세스++
    show_process += 1
    # 진행사항 보여주기
    clear_output()
    print("Process : ", total_audio_num, "/", show_process)
    
    #진행
    MakeSpectrogramImages(path, SLICE_VALUE, IMAGE_SIZE)
    
print("All Done")

Process :  5 / 5
Wish_upon_Twin_Stars step num :  1191
step :  0


D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\matplotlib\image.py:395: UserWarning: Warning: converting a masked element to nan.
  dv = (np.float64(self.norm.vmax) -
D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\matplotlib\image.py:396: UserWarning: Warning: converting a masked element to nan.
  np.float64(self.norm.vmin))
D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\matplotlib\image.py:403: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\matplotlib\image.py:408: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)
D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\matplotlib\colors.py:918: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
D:\Develop\Anaconda3\envs\keras_gpu\lib\site-packages\numpy\ma\core.py:715: UserWarning: Warning: converting a masked element to nan.
  data = np.array(a

step :  100
step :  200
step :  300
step :  400
step :  500
step :  600
step :  700
step :  800
step :  900
step :  1000
step :  1100
Wish_upon_Twin_Stars spectrograms done
All Done
